In [1]:
#---Torch
import torch
from torch.nn import Module, Conv2d, Linear, MaxPool2d, ReLU, LogSoftmax
from torch import flatten
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from sklearn.metrics import classification_report
from torch import nn
#---Others
import matplotlib.pyplot as plt
import numpy as np
import time 

In [10]:
#Before everything, define de device to use: GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Create a CNN with Pytorch

In [11]:
#Define the class. the class helps in the process of initilalizing variables and uso those viarable to make operations on inputs in a certain order
class LeNet(Module):
    def __init__(self, numChannels, classes):
        super(LeNet, self).__init__() #call parent constructor

        #1 set: Convolution-Relu-Max
        self.conv1 = Conv2d(in_channels= numChannels, out_channels= 20, kernel_size= (5,5))
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size= (2,2), stride= (2,2))

        #2 set: Convolution-Relu-Max
        self.conv2 = Conv2d(in_channels=20, out_channels=50, kernel_size=(5,5))
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2,2), stride=(2,2))

        #3 set: Fully Connected layer - Relu
        self.fc1 = Linear(in_features=800, out_features=500)
        self.relu3 = ReLU()

        #4 set: Fully connected - softmax classifier
        self.fc2 = Linear(in_features=500,out_features=classes)
        self.logSoftmax = LogSoftmax(dim=1)
    
#---Define the steps of the NN
    def forward(self, x):
        #pass the input x through all the variables
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)

        x = self.fc2(x)
        output = self.logSoftmax(x)

        return output

### Load the dataset and split it

In [17]:
trainData = KMNIST(root="data", train=True, download=True, transform=ToTensor())
testData = KMNIST(root="data", train=False, download=True, transform=ToTensor())

#Get the validation dataset
num_train_s = int(len(trainData)*0.75)
num_val_s = int(len(trainData)*0.25)
(trainData, valData) = random_split(trainData, [num_train_s, num_val_s],generator=torch.Generator().manual_seed(42))

### Configure the pipeline using DataLoader

In [18]:
trainDataLoader = DataLoader(trainData,shuffle=True, batch_size=64)
valDataLoader = DataLoader(valData, shuffle=True, batch_size=64)
testDataLoader = DataLoader(testData, batch_size=64) #Remember no shuffle on test 

In [6]:
#Calculate steps per epoch
trainSteps = len(trainDataLoader.dataset)//64
valSteps = len(valDataLoader.dataset)//64

### Initialize the model and components

In [7]:
model = LeNet(numChannels=1, classes=len(trainData.dataset.classes)).to(device)
opt = Adam(model.parameters(), lr=1e-3)
lossFn = nn.NLLLoss()

#Get dictinary to store all trainig history
H = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": []
}

In [8]:
model.parameters

<bound method Module.parameters of LeNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (logSoftmax): LogSoftmax(dim=1)
)>

### define the train and test process

In [15]:
startTime = time.time()

for e in range(0, 10):
    model.train() #Put the model on train mode

    #Initializing the total training and validation
    totalTrainLoss = 0
    totalValLoss = 0

    trainCorrect = 0
    valCorrect = 0

    for (x, y) in trainDataLoader:
        (x,y) = (x.to(device), y.to(device))

        #Do forward pass and calculate training loss
        pred = model(x)
        loss = lossFn(pred, y)

        #define the 3 most important steps

        opt.zero_grad()
        loss.backward()
        opt.step()

        #Add the train loss and accuracy
        totalTrainLoss += loss
        trainCorrect += (pred.argmax(1)==y).type(torch.float).sum().item()

        #Switch off autograd with .no-grad
    with torch.no_grad():

        model.eval() #and put the model on evaluation mode

        for (x,y) in valDataLoader:
            (x, y) = (x.to(device), y.to(device))
            pred = model(x)
            totalValLoss += lossFn(pred, y)
            valCorrect += (pred.argmax(1)==y).type(torch.float).sum().item()

    #calculate the training and val loss average
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps

    #calculate accuracy

    trainCorrect = trainCorrect/len(trainDataLoader.dataset)
    valCorrect = valCorrect/len(valDataLoader.dataset)

    #Update training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy()) #due to the backward pass
    H["train_acc"].append(trainCorrect)
    H["val_loss"].append(avgValLoss.cpu().detach().numpy()) #due to the backward pass
    H["val_acc"].append(trainCorrect)

    print("[INFO] EPOCH: {}/{}".format(e + 1, 10))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avgTrainLoss, trainCorrect))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(avgValLoss, valCorrect))


endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

[INFO] EPOCH: 1/10
Train loss: 0.058191, Train accuracy: 0.9817
Val loss: 0.083894, Val accuracy: 0.9735

[INFO] EPOCH: 2/10
Train loss: 0.035911, Train accuracy: 0.9894
Val loss: 0.081237, Val accuracy: 0.9779

[INFO] EPOCH: 3/10
Train loss: 0.029287, Train accuracy: 0.9909
Val loss: 0.086470, Val accuracy: 0.9759

[INFO] EPOCH: 4/10
Train loss: 0.023115, Train accuracy: 0.9925
Val loss: 0.076216, Val accuracy: 0.9799

[INFO] EPOCH: 5/10
Train loss: 0.014633, Train accuracy: 0.9953
Val loss: 0.075994, Val accuracy: 0.9812

[INFO] EPOCH: 6/10
Train loss: 0.014411, Train accuracy: 0.9954
Val loss: 0.086828, Val accuracy: 0.9800

[INFO] EPOCH: 7/10
Train loss: 0.012144, Train accuracy: 0.9961
Val loss: 0.105381, Val accuracy: 0.9766

[INFO] EPOCH: 8/10
Train loss: 0.010482, Train accuracy: 0.9964
Val loss: 0.079974, Val accuracy: 0.9829

[INFO] EPOCH: 9/10
Train loss: 0.008616, Train accuracy: 0.9973
Val loss: 0.078090, Val accuracy: 0.9833

[INFO] EPOCH: 10/10
Train loss: 0.004563, Trai

### Evaluate on test set

In [27]:
with torch.no_grad():
    model.eval()

    #Store predictions
    preds = []

    for (x, y) in testDataLoader:
        x = x.to(device) #don't forget to pass it to GPU/CUDA

        #make predictions
        pred = model(x)
        preds.extend(pred.argmax(axis=1).cpu().numpy()) #

        #Classification report
print(classification_report(testData.targets.cpu().numpy(), np.array(preds), target_names=testData.classes)) #remember to put everything as a numpy array


              precision    recall  f1-score   support

           o       0.96      0.94      0.95      1000
          ki       0.91      0.93      0.92      1000
          su       0.89      0.94      0.92      1000
         tsu       0.95      0.97      0.96      1000
          na       0.93      0.93      0.93      1000
          ha       0.97      0.93      0.95      1000
          ma       0.95      0.97      0.96      1000
          ya       0.98      0.83      0.90      1000
          re       0.96      0.93      0.95      1000
          wo       0.86      0.98      0.92      1000

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000

